# 🤖 Reze AI - Ollama 모델 학습
Google Colab에서 Ollama 모델을 학습하고 라즈베리파이로 배포

## 사전 준비
1. GitHub 리포지토리에 Modelfile 업로드
2. Colab Runtime을 GPU로 변경 (Runtime → Change runtime type → T4 GPU)

In [ ]:
# 1. Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# Ollama 서버 백그라운드 실행
import subprocess
import time

# Ollama 서버 시작
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)  # 서버 시작 대기
print("✅ Ollama 서버 시작됨")

In [ ]:
# 2. 베이스 모델 다운로드
!ollama pull llama3.2:1b
print("✅ 베이스 모델 다운로드 완료")

In [ ]:
# 3. GitHub에서 Modelfile 가져오기
!git clone https://github.com/KhankhanLee/reze.git
!cd reze && git pull origin main
print("✅ 리포지토리 클론 완료")

In [ ]:
# 4. 커스텀 모델 생성 (학습)
!cd reze && ollama create reze -f Modelfile.example
print("✅ 커스텀 모델 'reze' 생성 완료!")

In [ ]:
# 5. 모델 테스트
!ollama run reze "안녕 레제! 오늘 기분이 어때?"

In [ ]:
# 6. Modelfile 추출 (학습된 설정 저장)
!ollama show reze --modelfile > reze_trained.Modelfile
print("✅ Modelfile 저장됨: reze_trained.Modelfile")

# 파일 내용 확인
!cat reze_trained.Modelfile

## 📥 라즈베리파이로 전송

**방법 1: Modelfile만 전송 (추천)**
```bash
# Colab에서 다운로드
from google.colab import files
files.download('reze_trained.Modelfile')
```

**방법 2: GitHub 커밋**
```bash
# Colab에서
!cd reze && cp ../reze_trained.Modelfile Modelfile.example
!cd reze && git add Modelfile.example
!cd reze && git commit -m "Update trained model"
# 이후 수동으로 push (토큰 필요)
```

In [ ]:
# Modelfile 다운로드
from google.colab import files
files.download('reze_trained.Modelfile')

## 🥧 라즈베리파이 배포

SSH로 라즈베리파이 접속 후:

```bash
# 1. Modelfile 업로드
scp reze_trained.Modelfile pi@raspberrypi:/home/pi/

# 2. 라즈베리파이에서 모델 생성
ssh pi@raspberrypi
ollama create reze -f reze_trained.Modelfile

# 3. 테스트
ollama run reze "안녕!"

# 4. 서비스 재시작
sudo systemctl restart ollama
```